# System-Versioned Temporal Tables

[Referenz Temporal Tables](https:\learn.microsoft.com\en-us\sql\relational-databases\tables\temporal-tables?view=sql-server-ver16)  

  

Temporal Tables erfassen Änderungen an Datensätzen, sodass es zum Beispiel möglich wird, Werte für einen bestimmten Zeitpunkt in der Vergangenheit abzufragen.

![Temporal Table](../_Images/temporal-querying.svg)  

Dieses Feature ist seit SQL Server 2016 verfügbar. Eine neue Tabelle kann bei Ihrer Erstellung eine History Tabelle zugewiesen bekommen, oder dies kann nachträglich für vorhandene Tabellen geschehen.

## Tabelle **Angebot**

Die Tabelle **Angebot**  hat eine History-Tabelle. Änderungen werden über die View **vw\_Angebot** vorgenommen. Ein Trigger trägt den User ein, der die Änderung am Datensatz gemacht hat.

In [ ]:
-- * übergeht die versteckten Spalten GueltigVon und GueltigBis
SELECT * FROM Angebot WHERE ID = 1;

In [ ]:
-- Diese Abfrage zeigt die aktuellen Werte des Datensatzes an
SELECT ID, Gegenstand, Mindestgebot, Gebot, Nutzer, GueltigVon, GueltigBis FROM Angebot WHERE ID = 1;

In [ ]:
-- Diese Abfrage zeigt die gesamte Historie des des Datensatzes an
SELECT ID, Gegenstand, Mindestgebot, Gebot, Nutzer, GueltigVon, GueltigBis 
FROM Angebot FOR SYSTEM_TIME ALL 
WHERE ID = 1
ORDER BY GueltigVon DESC;

In [ ]:
-- Beispiel Abfrage für einen bestimmten Zeitpunkt
SELECT ID, Gegenstand, Mindestgebot, Gebot, Nutzer, GueltigVon, GueltigBis 
FROM Angebot FOR SYSTEM_TIME AS OF '2020-03-04 15:07' 
WHERE ID = 1
ORDER BY GueltigVon DESC;

In [ ]:
-- Ein neues Gebot wird abgegeben (Werte müssen >=10 über dem letzten Gebot liegen)
UPDATE vw_Angebot SET Gebot = 85 WHERE ID = 1;

In [ ]:
-- Auch die Tabelle Produkt hat eine Historie
-- Frage die Historie des Produkts mit der ProduktID 9 ab



## Nachträglich einen Temporal Table zuweisen

In [ ]:
BEGIN TRAN;

ALTER TABLE dbo.Kunde ADD
  GueltigVon DATETIME2(3) GENERATED ALWAYS AS ROW START HIDDEN NOT NULL
    CONSTRAINT dft_Kunde_GueltigVon DEFAULT('19000101'),
  GueltigBis DATETIME2(3) GENERATED ALWAYS AS ROW END HIDDEN NOT NULL
    CONSTRAINT dft_Kunde_GueltigBis DEFAULT('99991231 23:59:59.999'),
  PERIOD FOR SYSTEM_TIME (GueltigVon, GueltigBis);

ALTER TABLE dbo.Kunde
  SET ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.KundeHistory ) );

ALTER TABLE dbo.Kunde DROP CONSTRAINT dft_Kunde_GueltigVon, dft_Kunde_GueltigBis;

COMMIT TRAN;

In [ ]:
-- Ändere die Telefonnummer eines Kunden und Frage die Historie des Datensatzes ab



## Neue Tabelle als Temporal Table erstellen

In [ ]:
CREATE TABLE Highscores (
    ID INT IDENTITY(1,1) NOT NULL
        CONSTRAINT pk_Highscores PRIMARY KEY(ID),
    Spieler VARCHAR(100) NOT NULL,    
    Spiel VARCHAR(100) NOT NULL,
    Score BIGINT NOT NULL,
    GueltigVon DATETIME2(3) GENERATED ALWAYS AS ROW START HIDDEN NOT NULL,
    GueltigBis DATETIME2(3) GENERATED ALWAYS AS ROW END HIDDEN NOT NULL,
    PERIOD FOR SYSTEM_TIME (GueltigVon, GueltigBis)    
)
WITH ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.HighscoresHistory ) );

In [ ]:
-- Um einen Temporal Table zu löschen, muss zuvor die Versionierung ausgeschaltet werden
ALTER TABLE Highscores SET
(SYSTEM_VERSIONING = OFF)
GO

DROP TABLE IF EXISTS Highscores;